### PRUEBA CON EL MODELO FINAL PARA LA PUESTA EN PRODUCCION CON LOS DATOS DE TEST NO UTILIZADOS HASTA AHORA



In [1]:
import pandas as pd
import numpy as np
import joblib
import sys
sys.path.append("C:/Users/Fernando/Desktop/Proyecto_Final_ML/src")
import preprocess_functions as pr
import funciones_clases_pipeline_produccion

In [2]:
data_test = pd.read_csv("C:/Users/Fernando/Desktop/Proyecto_Final_ML/data/data_test.csv", index_col=0)

In [3]:
# MODIFICACION DE LA COLUMNA Y (target)
data_test = data_test[~data_test["Company response"].isin(["In progress", "Untimely response"])]
# Creación del diccionario con los valores para codificar la columna Company reponse.
estado_respuesta = {'Closed with monetary relief': 0, 
       'Closed with non-monetary relief':0, 'Closed with explanation': 1, 'Closed': 1}

data_test["Company response"] = data_test["Company response"].map(estado_respuesta)
X = data_test.drop(columns=["Company response"])
y = data_test["Company response"]



In [4]:
# Cargamos el pipeline

pipeline = joblib.load("C:/Users/Fernando/Desktop/Proyecto_Final_ML/pipelines/pipeline_produccion.pkl")

In [5]:
y_pred = pipeline.predict(X)
y_proba = pipeline.predict_proba(X)

In [6]:
from sklearn.metrics import classification_report, roc_auc_score
print(classification_report(y, y_pred))
print(roc_auc_score(y, y_proba[:, 1]))

              precision    recall  f1-score   support

           0       0.28      0.75      0.41      1070
           1       0.88      0.49      0.63      4000

    accuracy                           0.54      5070
   macro avg       0.58      0.62      0.52      5070
weighted avg       0.75      0.54      0.58      5070

0.6420413551401869


## Modelo muy robusto ya que los valores en las métricas obtenidos para el modelo con los datos de validacion no expuestos anteriormente presentan valores similares a los de entrenamiento y test

## DESPLIEGUE CON GRADIO

In [7]:
import joblib
import gradio as gr
import pandas as pd

In [8]:
# Guardado de las 10 primeras filas de la partición de datos de test

X_produccion = X.iloc[:11, :].to_csv("valores_para_testeo_gradio.csv", index=False)

In [ ]:
import joblib
import gradio as gr
import pandas as pd

pipeline = joblib.load("C:/Users/Fernando/Desktop/Proyecto_Final_ML/pipelines/pipeline_produccion.pkl")

def predecir_csv(ruta_csv):
    
    df = pd.read_csv(ruta_csv)

    predicciones = pipeline.predict(df)
    probabilidad = pipeline.predict_proba(df)

    prob_predicha = []
    for i, pred in enumerate(predicciones):
        prob_predicha.append(probabilidad[i, pred])
        
    df["Prediccion"] = predicciones
    df["Probabilidad"] = prob_predicha

    df["Mensaje"] = df["Prediccion"].map({
        0: "La queja requiere compensación",
        1: "La queja no requiere compensación"
    })

    return df


In [10]:
demo = gr.Interface(
    fn=predecir_csv,
    inputs=gr.File(type="filepath", file_types=[".csv"], label="Sube un CSV con las quejas"),
    outputs=gr.Dataframe(label="Predicciones"),
    title="Modelo predictivo de prioridad de atención a quejas"
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\Fernando\Desktop\Proyecto_Final_ML\.venv\Lib\site-packages\gradio\queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Fernando\Desktop\Proyecto_Final_ML\.venv\Lib\site-packages\gradio\route_utils.py", line 349, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Fernando\Desktop\Proyecto_Final_ML\.venv\Lib\site-packages\gradio\blocks.py", line 2274, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Fernando\Desktop\Proyecto_Final_ML\.venv\Lib\site-packages\gradio\blocks.py", line 1781, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Fernando\Desktop\Proyecto_Final_ML\.venv\